# Interpolate CROCO variables onto particles position with xarray

**WARNING**

This is a very basic notebook not meant to deal with every possible Pyticles
experiment. This is for 3D experiment only. 

- CROCO variables are loaded/computed using R_vars.py,
see `croco_vars-r-tools.ipynb` for more information.

- 
Results are saved in a separate netcdf file

subtime interpolation of particles are not coded yet.
Use linear function from add_var_to_file.py, this script also
allow to save results in Pyticles Netcdf file.

In [1]:
# ----------------------------------------------------------------------------
import time
import os
import sys
import warnings

import numpy as np
import numpy.ma as ma
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import xarray as xr

# pyticles related modules and tools
sys.path.append("../../Modules/")
import pyticles_sig_sa as part
import R_vars as va
from R_files import load


In [4]:
def tight_coord(px, py):
    "Smaller CROCO domain that contains particles"
    coord = np.array([np.floor(np.nanmin(py)), np.ceil(np.nanmax(py)),
                np.floor(np.nanmin(px)), np.ceil(np.nanmax(px))], dtype=int)
    return coord



In [2]:
from dask.distributed import Client, progress

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 4.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45607,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 4.00 GiB
Comm: tcp://127.0.0.1:38614,Total threads: 1
Dashboard: http://127.0.0.1:38885/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:39040,


In [5]:
# --> inputs
part_path = '/home/datawork-lemar-apero/jcollin/pyticles/cfl/'
partfile = part_path + 'uncompressed_substep-cfl0.1-bis_28_3800.nc'
my_simul = 'polgyr'

# --> ouputs
save_data = False
varname = 'rho1'
varout = 'p' + varname

# --> I/O
# be carfeull with chunking, chunks shall be of 1 Mb size approximately
# Too small chunks leads to an increase of computing time due to overhead
# Too big chunks slow down process due to RAM saturation and poor parallelization
chunk_nq = 100
chunk_time = -1
ntimes = 2 # subsample time for example (use 

# ------------------------------------------------------------------------
# --> open Pyticles file with chunks to spare memory
ds = xr.open_dataset(    
    partfile,
    chunks={
        "nq": chunk_nq,
        "time": chunk_time,
    },
)

ptimes = ds.time.data
parameters = my_simul + ' [0,10000,0,10000,[1,100,1]] '+ format(int(ptimes[0]))
simul = load(simul = parameters, floattype=np.float64)
nx, ny, nz = [simul.coord[1], simul.coord[3], simul.coord[4].size]

# --> time loop
for ptime in ptimes[:ntimes]:
    print('time step = {}'.format(ptime))
    # synchronize CROCO fields to Pytciles outputs
    simul.update(np.round(ptime))
    
    # --> get smallest domain with particles (fast loading)
    t0 = time.time()
    px = ds.px.sel(time=ptime).persist()
    py = ds.py.sel(time=ptime).persist()
    pz = ds.pz.sel(time=ptime).persist()
    coord = tight_coord(px, py)
    print(f"--> time to load particles position {time.time() - t0} s")

    # --> get 3D CROCO variable
    t0 = time.time()
    var =  xr.DataArray(
            va.var(varname, simul, method = 'new', coord=coord).data,
            dims=("xi", "eta", "sigma"),
            name=varname,
    )
    print(f"--> time to load {varname} {time.time() - t0} s")

    # --> Interpolate onto particles position
    t0 = time.time()
    pvar = part.map_var(simul, var, px, py, pz, ng=1, coord=coord)   
    print(f"--> time to interpolate {varname} {time.time() - t0} s")

simulname is None
args (['polgyr', '[0,10000,0,10000,[1,100,1]]', '3800'],)
args[0] ['polgyr', '[0,10000,0,10000,[1,100,1]]', '3800']
len(args[0]) 3
time of simulation is: 3800
coord
loading /home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc
loading /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.03800.nc
get domain [0,10000,0,10000,[1,100,1]] [0,10
coordmax
loading /home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc
loading /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.03800.nc
get domain [0,1e9,0,1e9,[1,1e9,1]] [0,1e
cst
read Cs_r in ncfile.Cs_r
dt
dt is read in  /home/datawork-lops-osi/mlecorre/POLGYR/HIS/polgyr_his.03800.nc
[0, 1602, 0, 2002, array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70,